# Inference Pipeline

When running this pipeline we need to set two important parameters:
- s3_input_path: location of the data to be used for inferencing
- model_arn: arn of the model to be used

In [ ]:
import os
import time
import boto3
import pandas as pd
import sagemaker.session
from datetime import datetime
from sagemaker import (
    image_uris,
    model_uris,
    script_uris,
    hyperparameters,
    get_execution_role,
)
from sagemaker.utils import name_from_base
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterBoolean,
)
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.steps import (
    ProcessingStep,
    TuningStep,
    TransformStep,
    CreateModelStep,
)
from sagemaker.workflow.step_collections import RegisterModel, EstimatorTransformer
from sagemaker.workflow.parameters import ParameterString
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor, ScriptProcessor
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import (
    DataCaptureConfig,
    DatasetFormat,
    DefaultModelMonitor,
)
from sagemaker.s3 import S3Uploader, S3Downloader
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
    HyperparameterTuner,
)
from sagemaker.workflow.functions import Join
from sagemaker.model import Model, ModelPackage
from sagemaker.inputs import TrainingInput, TransformInput, BatchDataCaptureConfig
from sagemaker.transformer import Transformer

## Pipeline parameters

In [ ]:
s3_bucket = ParameterString(
    "s3_bucket", default_value="s3://sagemaker-eu-west-1-708699854342"
)
s3_project_path = ParameterString(
    "s3_project_path", default_value="knnights/test/shapeshifter"
)

model_location = ParameterString(
    "model_location",
    default_value="s3://sagemaker-eu-west-1-708699854342/shapeshifter/models/lightgbm/zz4f6n0k2v79-LightGB-oJZZOAXMXV-001-2a98dea5/output/model.tar.gz",
)
s3_input_data = ParameterString(
    "s3_input_data",
    default_value="s3://nike--708699854342--test--eu-west-1/knnights/test/shapeshifter/inference/data_pipeline",
)
encoders_path = ParameterString(
    "encoders_path",
    default_value="s3://sagemaker-eu-west-1-708699854342/shapeshifter/data/processed/2023-01-10T07:47:47.735Z/encoders/",
)

In [ ]:
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = s3_bucket

pipeline_session = PipelineSession()

cache_config = CacheConfig(enable_caching=False, expire_after="1d")

# S3 paths
project = "shapeshifter"
model_type = "lightgbm"

project_path = Join(values=["s3:/", default_bucket, s3_project_path], on="/")

inference_path = Join(values=[project_path, "inference"], on="/")
inference_live_path = Join(values=[inference_path, "live"], on="/")
inference_input_path = Join(values=[inference_path, "input"], on="/")
data_capture = Join(values=[project_path, "data_capture"], on="/")

## Get data

In [ ]:
sklearn_processor = SKLearnProcessor(
    framework_version="1.0-1",
    role=role,
    instance_type="ml.m5.2xlarge",
    instance_count=1,
    sagemaker_session=pipeline_session,
)

root = "/opt/ml/processing"

step_process = ProcessingStep(
    name="ShapeshifterPrepInferenceData",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            source=s3_input_data, destination=Join(values=[root, "/input/data"])
        ),
        ProcessingInput(
            source=encoders_path, destination=Join(values=[root, "/input/encoders"])
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="inference_data",
            source=f"{root}/output",
            destination=inference_input_path,
        ),
    ],
    code="inference_processing.py",
    cache_config=cache_config,
)

## Batch Transform

<div class="alert alert-info"> 💡 <strong> Input data </strong>

Make sure to use live data instead of test data in the final inference pipeline!
</div>

In [ ]:
# Step 3: Create a model package
# lgbm_model = ModelPackage(
#     role=role,
#     model_package_arn=boto_client.list_model_packages(
#         ModelPackageGroupName=f"{project}-{model_type}",
#         ModelApprovalStatus="Approved"
#     )["ModelPackageSummaryList"][0]["ModelPackageArn"],
#     source_dir="model_scripts",
#     entry_point="inference.py",
#     sagemaker_session=pipeline_session,
# )

docker_image_inference = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=f"{model_type}-regression-model",
    model_version="*",
    image_scope="inference",
    instance_type="ml.m5.large",
)

lgbm_model = Model(
    model_data=model_location,
    image_uri=docker_image_inference,
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_best_lightgbm = ModelStep(
    name="GetBestLightGBM",
    step_args=lgbm_model.create(instance_type="ml.m5.large"),
)

lightgbm_transformer = Transformer(
    model_name=step_create_best_lightgbm.properties.ModelName,
    instance_type="ml.m5.large",
    instance_count=1,
    output_path=inference_live_path,
    sagemaker_session=pipeline_session,
    assemble_with="Line",
    accept="text/csv",
    strategy="MultiRecord",
    max_payload=2,
)

step_lightgbm_transform = TransformStep(
    name="LightGBMTransform",
    transformer=lightgbm_transformer,
    inputs=TransformInput(
        step_process.properties.ProcessingOutputConfig.Outputs[
            "inference_data"
        ].S3Output.S3Uri,
        content_type="text/csv",
        split_type="Line",
        batch_data_capture_config=BatchDataCaptureConfig(
            destination_s3_uri=data_capture,
            generate_inference_id=True,
        ),
    ),
)

## Construct pipeline

In [ ]:
pipeline_name = f"ShapeshifterPipeline-Inference"
pipeline = Pipeline(
    name=pipeline_name,
    steps=[step_process, step_create_best_lightgbm, step_lightgbm_transform],
    parameters=[
        s3_bucket,
        s3_project_path,
        s3_input_data,
        encoders_path,
        #         model_arn,
        model_location,
    ],
)

pipeline.upsert(role_arn=role)

In [ ]:
# execution = pipeline.start(execution_display_name=f"{project}-{round(time.time())}")